### 787. Cheapest Flights Within K Stops
Medium

There are n cities connected by some number of flights. You are given an array flights where flights[i] = [fromi, toi, pricei] indicates that there is a flight from city fromi to city toi with cost pricei.

You are also given three integers src, dst, and k, return the cheapest price from src to dst with at most k stops. If there is no such route, return -1.

 

Example 1:


Input: n = 4, flights = [[0,1,100],[1,2,100],[2,0,100],[1,3,600],[2,3,200]], src = 0, dst = 3, k = 1
Output: 700
Explanation:
The graph is shown above.
The optimal path with at most 1 stop from city 0 to 3 is marked in red and has cost 100 + 600 = 700.
Note that the path through cities [0,1,2,3] is cheaper but is invalid because it uses 2 stops.


Example 2:
Input: n = 3, flights = [[0,1,100],[1,2,100],[0,2,500]], src = 0, dst = 2, k = 1
Output: 200
Explanation:
The graph is shown above.
The optimal path with at most 1 stop from city 0 to 2 is marked in red and has cost 100 + 100 = 200.

Example 3:
Input: n = 3, flights = [[0,1,100],[1,2,100],[0,2,500]], src = 0, dst = 2, k = 0
Output: 500
Explanation:
The graph is shown above.
The optimal path with no stops from city 0 to 2 is marked in red and has cost 500.
 

Constraints:

1 <= n <= 100
0 <= flights.length <= (n * (n - 1) / 2)
flights[i].length == 3
0 <= fromi, toi < n
fromi != toi
1 <= pricei <= 104
There will not be any multiple flights between two cities.
0 <= src, dst, k < n
src != dst


### Depth-First-Search with Memoization

We have a dedicated start and endpoint. We have a bunch of choices for each node in the form of its neighbors. And, we want to minimize the overall shortest distance from the source to the destination which can be represented as a recursive structure in terms of shortest distances of its neighbors to the destination. So, we can apply a dynamic programming approach to solve this problem. 

As with any recursive approach, we need to figure out the state of recursion. There are two parameters here which will control our recursion. One is obviously the node itself. The other is the number of steps. 

### Algorithm

1. We'll define a function called recurse which will take two inputs: node and stops.

2. We'll also define a dictionary memo of tuples that will store the optimal solution for each recursion state encountered.

3. At each stage, we'll first check if we have reached the destination or not. If we have, then no more moves have to be made and we return a value of 0 since the destination is at a zero distance from itself.

4. Next, we check if we have any more stops left. If we don't then we return inf basically representing that we cannot reach the destination from the current recursion state.

5. Finally, we check if the current recursion state is cached in the memo dictionary and if it is, we return the answer right away.

6. If none of these conditions are met,we progress in our recursion. For that we will iterate over the adjacency matrix to obtain the neighbors for the current node and make a recursive call for each one of them. The node would be the neighboring node and the number of stops would incremeneted by 1.

7. To each of these recursion calls, we add the weight of the corresponding edge i.e.
- recurse(neighbor, stops + 1) + weight(node, neighbor)

8. We need to return the result of recurse(src, 0) as the answer.

In [9]:
from collections import defaultdict

class Solution:
    def __init__(self):
        self.adj_matrix = None
        self.memo = defaultdict()
    def recurse(self, node, stops,dst,n):
        # No need to go any further if the destination is reached    
        if node == dst:
            return 0
        # Can't go any further if no stops left
        if stops < 0:
            return float('inf')
        # If the result of this state is already cached, return it
        if (node,stops) in self.memo:
            return self.memo[(node,stops)]
        # Recursive calls over all the neighbors
        ans = float('inf')
        for nei in range(n):
            if self.adj_matrix[node][nei] > 0:
                ans = min(ans, self.recurse(nei,stops-1,dst,n) + self.adj_matrix[node][nei])
        # Cache the result
        self.memo[(node,stops)] = ans
        return self.memo[(node,stops)]

    def findCheapestPrice(self, n: int, flights: list[list[int]], src: int, dst: int, k: int) -> int:
        self.adj_matrix = [[0 for _ in range(n)] for _ in range(n)]
        self.memo = {}

        for s, d, w in flights:
            self.adj_matrix[s][d] = w

        result = self.recurse(src,k, dst,n)

        return -1 if result == float('inf') else result

example = Solution()
print(example.findCheapestPrice(n = 4, flights = [[0,1,100],[1,2,100],[2,0,100],[1,3,600],[2,3,200]], src = 0, dst = 3, k = 1))
print(example.findCheapestPrice(n = 3, flights = [[0,1,100],[1,2,100],[0,2,500]], src = 0, dst = 2, k = 1))
print(example.findCheapestPrice(n = 3, flights = [[0,1,100],[1,2,100],[0,2,500]], src = 0, dst = 2, k = 0))

700
200
500


- Time Complexity: The time complexity for a recursive solution is defined by the number of recursive calls we make and the time it takes to process one recursive call. The number of recursive calls we can potentially make is O(V⋅K). In each recursive call, we iterate over a given node's neighbors. That takes time O(V) because we are using an adjacency matrix. Thus, the overall time complexity is O($V^2$ ⋅K).
- Space Complexity: O(V⋅K+$V^2$) where O(V⋅K) is occupied by the memo dictionary and the rest by the adjacency matrix structure we build in the beginning. 
- Runtime: 738 ms, faster than 5.90% of Python3 online submissions for Cheapest Flights Within K Stops.
- Memory Usage: 15.6 MB, less than 13.48% of Python3 online submissions for Cheapest Flights Within K Stops.

In [14]:
# Approach 1: Dijkstra's Algorithm
import heapq

class Solution:
    
    def findCheapestPrice(self, n: int, flights, src: int, dst: int, K: int) -> int:
        
        # Build the adjacency matrix
        adj_matrix = [[0 for _ in range(n)] for _ in range(n)]
        for s, d, w in flights:
            adj_matrix[s][d] = w
            
        # Shortest distances array
        distances = [float("inf") for _ in range(n)]
        current_stops = [float("inf") for _ in range(n)]
        distances[src], current_stops[src] = 0, 0
        
        # Data is (cost, stops, node)
        minHeap = [(0, 0, src)]     
        
        while minHeap:
            
            cost, stops, node = heapq.heappop(minHeap)
            
            # If destination is reached, return the cost to get here
            if node == dst:
                return cost
            
            # If there are no more steps left, continue 
            if stops == K + 1:
                continue
             
            # Examine and relax all neighboring edges if possible 
            for nei in range(n):
                if adj_matrix[node][nei] > 0:
                    dU, dV, wUV = cost, distances[nei], adj_matrix[node][nei]
                    
                    # Better cost?
                    if dU + wUV < dV:
                        distances[nei] = dU + wUV
                        heapq.heappush(minHeap, (dU + wUV, stops + 1, nei))
                        current_stops[nei] = stops
                    elif stops < current_stops[nei]:
                        #  Better steps?
                        heapq.heappush(minHeap, (dU + wUV, stops + 1, nei))
                        
        return -1 if distances[dst] == float("inf") else distances[dst]

example = Solution()
print(example.findCheapestPrice(n = 4, flights = [[0,1,100],[1,2,100],[2,0,100],[1,3,600],[2,3,200]], src = 0, dst = 3, K = 1))
print(example.findCheapestPrice(n = 3, flights = [[0,1,100],[1,2,100],[0,2,500]], src = 0, dst = 2, K = 1))
print(example.findCheapestPrice(n = 3, flights = [[0,1,100],[1,2,100],[0,2,500]], src = 0, dst = 2, K = 0)) 

700
200
500


- Time Complexity:  O($V^2$ * log V)
- Space Complexity: O($V^2$ ) is the overall space complexity. 
O(V) is occupied by the two dictionaries and also by the heap and $V^2$  by the adjacency matrix structure. 
- Runtime: 109 ms, faster than 96.43% of Python3 online submissions for Cheapest Flights Within K Stops.
- Memory Usage: 15.2 MB, less than 52.95% of Python3 online submissions for Cheapest Flights Within K Stops.

In [40]:
# dp solution
class Solution:
    def findCheapestPrice(self, n: int, flights, src: int, dst: int, k: int) -> int:
        memo = [[ float('inf') for _ in range(n)] for _ in range(k+2)]
        memo[0][src] = 0
        for i in range(1,k+2):
            memo[i][src] = 0
            for s, d, w in flights:
                print(s,d,w)
                memo[i][d] = min(memo[i][s] + w , memo[i][d])
                print('memo:',memo)
        return -1 if memo[k+1][dst] == float('inf') else memo[k+1][dst]

example = Solution()
print(example.findCheapestPrice(n = 4, flights = [[0,1,100],[1,2,100],[2,0,100],[1,3,600],[2,3,200]], src = 0, dst = 3, k = 1))
print(example.findCheapestPrice(n = 3, flights = [[0,1,100],[1,2,100],[0,2,500]], src = 0, dst = 2, k = 1))
print(example.findCheapestPrice(n = 3, flights = [[0,1,100],[1,2,100],[0,2,500]], src = 0, dst = 2, k = 0)) 

0 1 100
memo: [[0, inf, inf, inf], [0, 100, inf, inf], [inf, inf, inf, inf]]
1 2 100
memo: [[0, inf, inf, inf], [0, 100, 200, inf], [inf, inf, inf, inf]]
2 0 100
memo: [[0, inf, inf, inf], [0, 100, 200, inf], [inf, inf, inf, inf]]
1 3 600
memo: [[0, inf, inf, inf], [0, 100, 200, 700], [inf, inf, inf, inf]]
2 3 200
memo: [[0, inf, inf, inf], [0, 100, 200, 400], [inf, inf, inf, inf]]
0 1 100
memo: [[0, inf, inf, inf], [0, 100, 200, 400], [0, 100, inf, inf]]
1 2 100
memo: [[0, inf, inf, inf], [0, 100, 200, 400], [0, 100, 200, inf]]
2 0 100
memo: [[0, inf, inf, inf], [0, 100, 200, 400], [0, 100, 200, inf]]
1 3 600
memo: [[0, inf, inf, inf], [0, 100, 200, 400], [0, 100, 200, 700]]
2 3 200
memo: [[0, inf, inf, inf], [0, 100, 200, 400], [0, 100, 200, 400]]
400
0 1 100
memo: [[0, inf, inf], [0, 100, inf], [inf, inf, inf]]
1 2 100
memo: [[0, inf, inf], [0, 100, 200], [inf, inf, inf]]
0 2 500
memo: [[0, inf, inf], [0, 100, 200], [inf, inf, inf]]
0 1 100
memo: [[0, inf, inf], [0, 100, 200], [0, 1